In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 1. Imports
import os
import numpy as np
import pandas as pd
import librosa
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle

# 2. Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. Load Data
train_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv")
sample_submission = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv")

train_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/"
test_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/"

# 4. Load Wav2Vec2 Model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
model.eval()

# 5. Dataset
class GrammarAudioDataset(Dataset):
    def __init__(self, df, path, processor, is_train=True):
        self.df = df
        self.path = path
        self.processor = processor
        self.is_train = is_train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file = self.df.iloc[idx]["filename"]
        file_path = os.path.join(self.path, file)
        if not os.path.exists(file_path):
            print(f"Missing file: {file_path}")
            return None

        waveform, sr = torchaudio.load(file_path)

# Ensure mono audio
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)  # Convert to mono
        waveform = waveform.squeeze().numpy()
        resampled = librosa.resample(waveform, orig_sr=sr, target_sr=16000)


        return {
            "input": torch.tensor(resampled, dtype=torch.float),
            "label": float(self.df.iloc[idx]["label"]) if self.is_train else None
        }

# 6. Collate Function
def custom_collate_fn(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None

    inputs = [item["input"] for item in batch]
    padded_inputs = pad_sequence(inputs, batch_first=True)

    if batch[0]["label"] is not None:
        labels = [item["label"] for item in batch]
        return padded_inputs, torch.tensor(labels, dtype=torch.float)
    else:
        return padded_inputs

# 7. Attention-based Pooling (can replace with mean if needed)
def attn_pooling(hidden_states):
    # Simple mean pooling
    return hidden_states.mean(dim=1)

# 8. Feature Extraction
def extract_features(dataloader):
    features, labels = [], []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            if batch is None:
                continue
            if isinstance(batch, tuple):
                x, y = batch
                x = x.to(device)
                labels.extend(y.cpu().numpy())
            else:
                x = batch.to(device)

            hidden_states = model(x).last_hidden_state
            pooled = attn_pooling(hidden_states).cpu().numpy()
            features.extend(pooled)

    return np.array(features), np.array(labels) if labels else None

# 9. Prepare Train Data
train_dataset = GrammarAudioDataset(train_df, train_path, processor, is_train=True)
train_loader = DataLoader(train_dataset, batch_size=4, collate_fn=custom_collate_fn)

# Use pickle to cache features
if os.path.exists("train_features.pkl"):
    with open("train_features.pkl", "rb") as f:
        X_train, y_train = pickle.load(f)
else:
    X_train, y_train = extract_features(train_loader)
    with open("train_features.pkl", "wb") as f:
        pickle.dump((X_train, y_train), f)

# 10. Train Regressor
regressor = Ridge(alpha=1.0)
regressor.fit(X_train, y_train)
scores = cross_val_score(regressor, X_train, y_train, scoring='r2', cv=5)
print("CV R^2 Scores:", scores)

# 11. Evaluation
y_pred = regressor.predict(X_train)
pearson_corr = pearsonr(y_train, y_pred)[0]
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"Train Pearson Correlation: {pearson_corr:.4f}")
print(f"Train RMSE: {rmse:.4f}")

# 12. Visualize Predictions
plt.figure(figsize=(8,5))
sns.scatterplot(x=y_train, y=y_pred)
plt.xlabel("Actual Grammar Score")
plt.ylabel("Predicted Score")
plt.title("Predicted vs Actual Grammar Scores")
plt.grid(True)
plt.show()

# 13. Inference on Test Set
test_dataset = GrammarAudioDataset(test_df, test_path, processor, is_train=False)
test_loader = DataLoader(test_dataset, batch_size=4, collate_fn=custom_collate_fn)

X_test, _ = extract_features(test_loader)
test_preds = regressor.predict(X_test)

sample_submission["label"] = test_preds
sample_submission.to_csv("submission.csv", index=False)
print("✅ Submission saved as submission.csv")


In [ ]:
df_submission = pd.read_csv('submission.csv')
df_submission

In [ ]:
!pip install openai-whisper


In [ ]:
# 1. Imports
import os
import numpy as np
import pandas as pd
import librosa
import whisper
import xgboost as xgb
from scipy.stats import pearsonr, skew
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# 2. Paths
train_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv")
sample_submission = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv")

train_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/"
test_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/"

# 3. Whisper Embeddings + MFCC Extraction
def extract_features(df, audio_path, model):
    features = []
    for fname in tqdm(df['filename']):
        path = os.path.join(audio_path, fname)
        y, sr = librosa.load(path, sr=16000)

        # MFCC stats
        mfcc = librosa.feature.mfcc(y=y, sr=16000, n_mfcc=13)
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)
        mfcc_skew = skew(mfcc, axis=1)

        # Whisper embeddings (we use the encoder output from tiny or base model)
        audio = whisper.load_audio(path)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio).to(model.device)
        with torch.no_grad():
            embedding = model.encoder(mel.unsqueeze(0))  # shape: (1, frames, dim)
            whisper_feat = embedding[0].mean(dim=0).cpu().numpy()

        # Combine
        feature_vector = np.hstack([mfcc_mean, mfcc_std, mfcc_skew, whisper_feat])
        features.append(feature_vector)

    return np.array(features)

# 4. Load Whisper Model
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
whisper_model = whisper.load_model("base", device=device)  # You can try "small" or "medium"

# 5. Extract Features
X_train = extract_features(train_df, train_path, whisper_model)
y_train = train_df["label"].values
X_test = extract_features(test_df, test_path, whisper_model)

# 6. Handle NaN/Inf
for arr in [X_train, X_test]:
    arr[np.isnan(arr)] = 0
    arr[np.isinf(arr)] = 0

# 7. XGBoost Regressor
xgb_model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)

# 8. Evaluation
y_pred_train = xgb_model.predict(X_train)
pearson_corr = pearsonr(y_train, y_pred_train)[0]
rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"✅ Pearson Correlation (Train): {pearson_corr:.4f}")
print(f"✅ RMSE (Train): {rmse:.4f}")

# 9. Visualizations
plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_train, y=y_pred_train)
plt.xlabel("Actual Grammar Score")
plt.ylabel("Predicted Score")
plt.title("Predicted vs Actual (Train)")
plt.grid(True)
plt.show()

residuals = y_train - y_pred_train
plt.figure(figsize=(8, 4))
sns.histplot(residuals, kde=True)
plt.title("Residuals Distribution")
plt.xlabel("Residuals")
plt.grid(True)
plt.show()

# 10. Test Prediction
test_preds = xgb_model.predict(X_test)
sample_submission["label"] = test_preds
sample_submission.to_csv("submission_whisper_xgb.csv", index=False)
print("📁 Submission saved as submission_whisper_xgb.csv")
